# Enriching EmDraCor metadata

In [1]:
# imports
import pandas as pd
import numpy as np
import requests
import re
import json
import time
import tqdm
!pip install urllib.requests
from urllib.request import urlopen
from statistics import mean
from sklearn import preprocessing

ERROR: Could not find a version that satisfies the requirement urllib.requests (from versions: none)
ERROR: No matching distribution found for urllib.requests

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


## 1. Load corpus metadata

In [2]:
# load basic metadata csv from github

url = "https://raw.githubusercontent.com/lucagiovannini7/emdracor/main/metadata/EmDraCor_metadata.csv"
metadata_table = pd.read_csv(url)
metadata_table

,name,id,wikidataId,firstAuthor,numOfCoAuthors,title,subtitle,normalizedGenre,digitalSource,originalSourcePublisher,...,numEdges,yearWritten,numOfSegments,wikipediaLinkCount,numOfActs,wordCountText,wordCountSp,wordCountStage,numOfP,numOfL
0,abeille-argelie,fre000001,Q63215430,Abeille,0,"Argélie, Reine de Thessalie",Tragédie,Tragedy,http://theatre-classique.fr/pages/programmes/e...,NaN,...,17,NaN,29,0.0,5,17307,15962,0,0,1884
1,alarcon-verdad-sospechosa,span######,NaN,Ruiz de Alarcón y Mendoza,0,La verdad sospechosa,NaN,Comedy,NaN,NaN,...,46,1619,41,NaN,3,16919,16461,473,1,3306
2,amenta-somiglianza,ita######,NaN,Amenta,0,La somiglianza,NaN,Comedy,https://books.google.de/books?id=nfwwhgTh1iYC,NaN,...,69,NaN,67,NaN,5,34634,33743,1242,2375,0
3,amescua-esclavo-demonio,span######,NaN,Mira de Amescua,0,El esclavo del demonio,NaN,NaN,NaN,NaN,...,220,1605,3,NaN,3,18153,17413,742,0,3516
4,andreini-lo-schiavetto,ita000020,Q83893270,Andreini,0,Lo schiavetto,NaN,Comedy,http://www.bibliotecaitaliana.it/testo/bibit00...,UTET,...,169,NaN,46,0.0,5,41485,38458,258,1659,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,weise-ungleich-liebesalliance,ger######,NaN,Weise,0,Ungleich und gleich gepaarte Liebes-Alliance,NaN,Comedy,https://books.google.de/books?id=xSJLAAAAcAAJ,NaN,...,360,NaN,90,NaN,5,47637,46801,968,1723,36
146,woodes-the-conflict-of-conscience,ep000799,NaN,Woodes,0,The Conflict of Conscience,NaN,Comedy,https://texts.earlyprint.org/works/A68918.xml,NaN,...,40,1572,16,NaN,6,21364,19186,60,0,1946
147,zamora-cada-uno,span######,NaN,Zamora,0,Cada uno es linaje aparte,y los Mazas de Aragón,NaN,https://books.google.de/books?id=6IXGXxO5iAgC,NaN,...,247,NaN,18,NaN,3,18240,16983,1406,0,3610
148,zamora-ser-fino-y-no-parecerlo,span######,NaN,Zamora,0,Ser fino y no parecerlo,NaN,NaN,https://books.google.de/books?id=6IXGXxO5iAgC,NaN,...,262,NaN,19,NaN,3,15998,15138,1141,0,3268


In [3]:
#print(list(metadata_table))

In [4]:
# drop unnecessary (= non numeric) columns

metadata_table = metadata_table.drop(columns=['wikidataId','firstAuthor', 'numOfCoAuthors', 'title', 'subtitle',
                                              'libretto', 'maxDegreeIds', 'yearPremiered', 'yearPrinted', 'yearWritten',
                                              'wikipediaLinkCount','originalSourceNumberOfPages','digitalSource',
                                              'originalSourcePublisher', 'originalSourcePubPlace','originalSourceYear'])
metadata_table = metadata_table.fillna(0)
metadata_table

,name,id,normalizedGenre,yearNormalized,size,averageClustering,density,averagePathLength,averageDegree,diameter,...,numOfPersonGroups,numConnectedComponents,numEdges,numOfSegments,numOfActs,wordCountText,wordCountSp,wordCountStage,numOfP,numOfL
0,abeille-argelie,fre000001,Tragedy,1674,8,0.822024,0.607143,1.392857,4.250000,2,...,0,1,17,29,5,17307,15962,0,0,1884
1,alarcon-verdad-sospechosa,span######,Comedy,1619,14,0.769965,0.505495,1.593407,6.571429,3,...,0,1,46,41,3,16919,16461,473,1,3306
2,amenta-somiglianza,ita######,Comedy,1706,13,0.903280,0.884615,1.115385,10.615385,2,...,0,1,69,67,5,34634,33743,1242,2375,0
3,amescua-esclavo-demonio,span######,0,1605,23,0.931742,0.869565,1.130435,19.130435,2,...,0,1,220,3,3,18153,17413,742,0,3516
4,andreini-lo-schiavetto,ita000020,Comedy,1612,24,0.859995,0.612319,1.391304,14.083333,3,...,1,1,169,46,5,41485,38458,258,1659,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,weise-ungleich-liebesalliance,ger######,Comedy,1708,51,0.707500,0.282353,2.069020,14.117647,4,...,0,1,360,90,5,47637,46801,968,1723,36
146,woodes-the-conflict-of-conscience,ep000799,Comedy,1572,18,0.705047,0.261438,1.741667,4.444444,3,...,0,3,40,16,6,21364,19186,60,0,1946
147,zamora-cada-uno,span######,0,1708,33,0.825678,0.467803,1.573864,14.969697,3,...,13,1,247,18,3,18240,16983,1406,0,3610
148,zamora-ser-fino-y-no-parecerlo,span######,0,1701,35,0.823315,0.440336,1.579832,14.971429,3,...,10,1,262,19,3,15998,15138,1141,0,3268


In [5]:
metadata_table.columns

Index(['name', 'id', 'normalizedGenre', 'yearNormalized', 'size',
       'averageClustering', 'density', 'averagePathLength', 'averageDegree',
       'diameter', 'datePremiered', 'maxDegree', 'numOfSpeakers',
       'numOfSpeakersFemale', 'numOfSpeakersMale', 'numOfSpeakersUnknown',
       'numOfPersonGroups', 'numConnectedComponents', 'numEdges',
       'numOfSegments', 'numOfActs', 'wordCountText', 'wordCountSp',
       'wordCountStage', 'numOfP', 'numOfL'],
      dtype='object')

In [6]:
# normalise language tags

def find_language(value):
	if "span" in value:
		return "ESP"
	elif "cal" in value:
		return "ESP"
	elif "ger" in value:
		return "GER"
	elif "ita" in value:
		return "ITA"
	elif "fre" in value:
		return "FRE"
	elif "ep" in value:
		return "ENG"

metadata_table['language'] = metadata_table['id'].map(find_language)
first_column = metadata_table.pop('language')
metadata_table.insert(1, 'language', first_column)
metadata_table = metadata_table.drop(columns=['id'])
display(metadata_table)

,name,language,normalizedGenre,yearNormalized,size,averageClustering,density,averagePathLength,averageDegree,diameter,...,numOfPersonGroups,numConnectedComponents,numEdges,numOfSegments,numOfActs,wordCountText,wordCountSp,wordCountStage,numOfP,numOfL
0,abeille-argelie,FRE,Tragedy,1674,8,0.822024,0.607143,1.392857,4.250000,2,...,0,1,17,29,5,17307,15962,0,0,1884
1,alarcon-verdad-sospechosa,ESP,Comedy,1619,14,0.769965,0.505495,1.593407,6.571429,3,...,0,1,46,41,3,16919,16461,473,1,3306
2,amenta-somiglianza,ITA,Comedy,1706,13,0.903280,0.884615,1.115385,10.615385,2,...,0,1,69,67,5,34634,33743,1242,2375,0
3,amescua-esclavo-demonio,ESP,0,1605,23,0.931742,0.869565,1.130435,19.130435,2,...,0,1,220,3,3,18153,17413,742,0,3516
4,andreini-lo-schiavetto,ITA,Comedy,1612,24,0.859995,0.612319,1.391304,14.083333,3,...,1,1,169,46,5,41485,38458,258,1659,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,weise-ungleich-liebesalliance,GER,Comedy,1708,51,0.707500,0.282353,2.069020,14.117647,4,...,0,1,360,90,5,47637,46801,968,1723,36
146,woodes-the-conflict-of-conscience,ENG,Comedy,1572,18,0.705047,0.261438,1.741667,4.444444,3,...,0,3,40,16,6,21364,19186,60,0,1946
147,zamora-cada-uno,ESP,0,1708,33,0.825678,0.467803,1.573864,14.969697,3,...,13,1,247,18,3,18240,16983,1406,0,3610
148,zamora-ser-fino-y-no-parecerlo,ESP,0,1701,35,0.823315,0.440336,1.579832,14.971429,3,...,10,1,262,19,3,15998,15138,1141,0,3268


In [7]:
# AFTER ENRICHMENT
# count how many without genre
#metadata_table.normalizedGenre.fillna('Other',inplace=True)  # still to fix, replace NaNs is not working
metadata_table.normalizedGenre.value_counts()
#df = metadata_table.loc[metadata_table['normalizedGenre'] == 0]
#df

# needs to further check English titles

Tragedy        57
Comedy         57
0              28
Tragicomedy     8
Name: normalizedGenre, dtype: int64

## 2. Metrics enrichment


In [8]:
# TEST -- get all type of useful metadata through the API
'''
metadata_url = "http://localhost:8088/api/corpora/em/play/virey-machabee"
response1 = urlopen(metadata_url)
metadata = json.loads(response1.read())

spoken_text_url = "http://localhost:8088/api/corpora/em/play/virey-machabee/spoken-text-by-character"
spoken_text = requests.get(spoken_text_url).json()

network_metrics_url = "http://localhost:8088/api/corpora/em/play/virey-machabee/metrics"
response2 = urlopen(network_metrics_url)
network_metrics = json.loads(response2.read())
'''
# this will be wrapped in a function

'\nmetadata_url = "http://localhost:8088/api/corpora/em/play/virey-machabee"\nresponse1 = urlopen(metadata_url)\nmetadata = json.loads(response1.read())\n\nspoken_text_url = "http://localhost:8088/api/corpora/em/play/virey-machabee/spoken-text-by-character"\nspoken_text = requests.get(spoken_text_url).json()\n\nnetwork_metrics_url = "http://localhost:8088/api/corpora/em/play/virey-machabee/metrics"\nresponse2 = urlopen(network_metrics_url)\nnetwork_metrics = json.loads(response2.read())\n'

#### 0. Adding centrality measures via API
Betweenness centrality, closeness centrality, eigenvector centrality

In [9]:
def get_centralities(network_metrics):

    from statistics import mean
    betweennesses = []
    closenesses = []
    eigenvectors = []
    for el in network_metrics['nodes']:
      betweennesses.append(el['betweenness'])
      closenesses.append(el['closeness'])
      eigenvectors.append(el['eigenvector'])
    avg_betweenness = mean(betweennesses)
    avg_closeness = mean(closenesses)
    avg_eigenvector = mean(eigenvectors)

    return avg_betweenness, avg_closeness, avg_eigenvector

In [10]:
# prepare empty columns

data = {'avg_betweenness': [],
        'avg_closeness': [],
        'avg_eigenvector': []}

extra_features = pd.DataFrame(data)
extra_features

,avg_betweenness,avg_closeness,avg_eigenvector


In [11]:
for slug in metadata_table['name']:
  url = f"http://localhost:8088/api/v1/corpora/em/plays/{slug}/metrics"
  print("Computing centrality measures for", url)
  response = requests.get(url)
  network_metrics = response.json()
  try:
    avg_betweenness, avg_closeness, avg_eigenvector = get_centralities(network_metrics)
  except Exception as e:
    print("This play throws an error:", url, e)

  new_row = {'avg_betweenness': avg_betweenness,
            'avg_closeness': avg_closeness,
            'avg_eigenvector': avg_eigenvector}
  extra_features = extra_features.append(new_row, ignore_index=True)

extra_features

Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/abeille-argelie/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/alarcon-verdad-sospechosa/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/amenta-somiglianza/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/amescua-esclavo-demonio/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/andreini-lo-schiavetto/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/anonym-lustig-pickelherings-spiel/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/anonym-macht-kleines-knaben/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/artieda-los-amantes/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/bailey-the-spiteful-sister

/tmp/ipykernel_29872/1656833532.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features = extra_features.append(new_row, ignore_index=True)
/tmp/ipykernel_29872/1656833532.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features = extra_features.append(new_row, ignore_index=True)
/tmp/ipykernel_29872/1656833532.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features = extra_features.append(new_row, ignore_index=True)
/tmp/ipykernel_29872/1656833532.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features = extra_features.append(new_row, ignore_index=True)
/tmp/ipykernel_29872/1656833532.py:1

Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/bregolini-pausania/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/campistron-juba/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/castro-mocedades-cid/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/cecchini-l-amico-tradito/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/centlivre-the-man-s-bewitch-d/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/cervantes-cerco-numancia/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/cicognini-mustafa/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/corneillep-rodogune/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/corneillet-mort-d-achille/metrics
Computin

/tmp/ipykernel_29872/1656833532.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features = extra_features.append(new_row, ignore_index=True)
/tmp/ipykernel_29872/1656833532.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features = extra_features.append(new_row, ignore_index=True)
/tmp/ipykernel_29872/1656833532.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features = extra_features.append(new_row, ignore_index=True)
/tmp/ipykernel_29872/1656833532.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features = extra_features.append(new_row, ignore_index=True)
/tmp/ipykernel_29872/1656833532.py:1

Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/della-porta-carbonaria/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/denham-the-sophy/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/diamante-hercules-ocana/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/dolce-marianna/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/du-ryer-clitophon/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/duffett-the-mock-tempest/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/fane-the-sacrifice/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/feinde-verwirrte-haus-jacob/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/fiorillo-la-lucilla-costante/metrics
Computing 

...

Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/suenos-hay-que-verdad-son/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/taille-corrivaus/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/tasso-aminta/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/tasso-intrichi-d-amore/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/testi-isola-alcina/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/tomkis-lingua/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/torelli-la-merope/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/tortoletti-gionata/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/tristan-parasite/metrics
Computing centrality measures for http://l

/tmp/ipykernel_29872/1656833532.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features = extra_features.append(new_row, ignore_index=True)
/tmp/ipykernel_29872/1656833532.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features = extra_features.append(new_row, ignore_index=True)
/tmp/ipykernel_29872/1656833532.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features = extra_features.append(new_row, ignore_index=True)
/tmp/ipykernel_29872/1656833532.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features = extra_features.append(new_row, ignore_index=True)
/tmp/ipykernel_29872/1656833532.py:1

Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/weise-masaniello/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/weise-ungleich-liebesalliance/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/woodes-the-conflict-of-conscience/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/zamora-cada-uno/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/zamora-ser-fino-y-no-parecerlo/metrics
Computing centrality measures for http://localhost:8088/api/v1/corpora/em/plays/zayas-traicion-amistad/metrics


/tmp/ipykernel_29872/1656833532.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features = extra_features.append(new_row, ignore_index=True)
/tmp/ipykernel_29872/1656833532.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features = extra_features.append(new_row, ignore_index=True)
/tmp/ipykernel_29872/1656833532.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features = extra_features.append(new_row, ignore_index=True)
/tmp/ipykernel_29872/1656833532.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features = extra_features.append(new_row, ignore_index=True)


,avg_betweenness,avg_closeness,avg_eigenvector
0,0.065476,0.741698,0.338143
1,0.049451,0.660847,0.243120
2,0.010490,0.904903,0.275634
3,0.006211,0.899261,0.205852
4,0.017787,0.739950,0.191526
...,...,...,...
145,0.021817,0.496874,0.113050
146,0.036356,0.462649,0.205194
147,0.018512,0.654649,0.161016
148,0.017571,0.647535,0.156253


In [12]:
table = pd.concat([metadata_table, extra_features],axis=1)
table.head()

,name,language,normalizedGenre,yearNormalized,size,averageClustering,density,averagePathLength,averageDegree,diameter,...,numOfSegments,numOfActs,wordCountText,wordCountSp,wordCountStage,numOfP,numOfL,avg_betweenness,avg_closeness,avg_eigenvector
0,abeille-argelie,FRE,Tragedy,1674,8,0.822024,0.607143,1.392857,4.250000,2,...,29,5,17307,15962,0,0,1884,0.065476,0.741698,0.338143
1,alarcon-verdad-sospechosa,ESP,Comedy,1619,14,0.769965,0.505495,1.593407,6.571429,3,...,41,3,16919,16461,473,1,3306,0.049451,0.660847,0.243120
2,amenta-somiglianza,ITA,Comedy,1706,13,0.903280,0.884615,1.115385,10.615385,2,...,67,5,34634,33743,1242,2375,0,0.010490,0.904903,0.275634
3,amescua-esclavo-demonio,ESP,0,1605,23,0.931742,0.869565,1.130435,19.130435,2,...,3,3,18153,17413,742,0,3516,0.006211,0.899261,0.205852
4,andreini-lo-schiavetto,ITA,Comedy,1612,24,0.859995,0.612319,1.391304,14.083333,3,...,46,5,41485,38458,258,1659,40,0.017787,0.739950,0.191526


#### 1. Adding metrics from Trilcke et al. 2017

In [13]:
#all-in index
def compute_all_in_index(metadata):
  all_in_index = metadata['allInIndex']
  return all_in_index

In [14]:
#final scene size
#last_segment = metadata['segments'][-1]['number']
def compute_final_scene_size(metadata):
  final_scene_size = len(metadata['segments'][-1]['speakers'])/len(metadata['characters'])
  return final_scene_size

In [15]:
''' [...] we introduced a measure we call the drama-change rate. The basis of our calculation is a modified Levenshtein distance, which only takes into account insertions ("add character") and deletions ("delete character").
In each case, we compare characters present in two consecutive scenes, eventually resulting in what we call the segment-change rate. The sum of all segment-change rates of a drama divided by the number of all
segment-change rates is what we call the drama-change rate '''

# segment change rate = no. of edits / total no. of characters

def compute_change_rate(metadata):

  num_edits = 0
  num_characters = 0
  segment_change_rate = []
  scene1 = 0
  scene2 = 1

  while scene2 < len(metadata['segments']):

    # 1. COUNTING DELETED SPEAKERS
    if 'speakers' in metadata['segments'][scene1]: # if speakers exist in 1st scene
      for speaker in metadata['segments'][scene1]['speakers']:
        if 'speakers' in metadata['segments'][scene2]:    # if speakers exist in 2nd scene
          if speaker not in metadata['segments'][scene2]['speakers']:
            num_edits += 1
        else:
            num_edits = len(metadata['segments'][scene1]['speakers'])  # all speakers have been deleted
    else:
       pass

    # 2. COUNTING ADDED SPEAKERS
    if 'speakers' in metadata['segments'][scene2]:
      for speaker in metadata['segments'][scene2]['speakers']:
        if 'speakers' in metadata['segments'][scene1]:
          if speaker not in metadata['segments'][scene1]['speakers']:
            num_edits += 1
        else:
            num_edits = len(metadata['segments'][scene2]['speakers']) # all speakers have been added
    else:
      pass

    #print("-------------Transition", scene1+1)
    #try:
    #  print("Speakers in Scene", scene1+1, metadata['segments'][scene1]['speakers'])
    #except:
    #  print("Speakers in Scene", scene1+1, 0)
    #try:
    #  print("Speakers in Scene", scene2+1, metadata['segments'][scene2]['speakers'])
    #except:
    #  print("Speakers in Scene", scene1+1, 0)
    #print("Number of edits (additions/deletions):", num_edits)

    if 'speakers' in metadata['segments'][scene1] and 'speakers' in metadata['segments'][scene2]:
        num_characters = len(list(set(metadata['segments'][scene1]['speakers'] + metadata['segments'][scene2]['speakers'])))
    else:
      try:
        num_characters = len(metadata['segments'][scene2]['speakers'])
      except:
        num_characters = len(metadata['segments'][scene1]['speakers'])

    #print("Number of speakers:", num_characters)
    segment_change_rate.append(num_edits/num_characters)
    #print("Segment change rate:", segment_change_rate)
    scene1 += 1
    scene2 += 1
    num_edits = 0
    num_characters = 0

  drama_change_rate = sum(segment_change_rate)/len(segment_change_rate)
  return drama_change_rate

In [66]:
# prepare empty columns

data = {'all_in_index': [],
        'final_scene_size': [],
        'drama_change_rate': []}

extra_features_2 = pd.DataFrame(data)
extra_features_2

,all_in_index,final_scene_size,drama_change_rate


In [17]:
# test
'''
url = "https://dracor.org/api/corpora/ger/play/weise-masaniello"
print(url)
response = requests.get(url)
metadata = json.loads(response.text)
drama_change_rate = compute_change_rate(metadata)
print(drama_change_rate)
'''

'\nurl = "https://dracor.org/api/corpora/ger/play/weise-masaniello"\nprint(url)\nresponse = requests.get(url)\nmetadata = json.loads(response.text)\ndrama_change_rate = compute_change_rate(metadata)\nprint(drama_change_rate)\n'

In [69]:
for slug in metadata_table['name']:
    url = "http://localhost:8088/api/v1/corpora/em/plays/" + slug
    print("Computing Trilcke et al. 2017 metrics for", url)
    response = requests.get(url)
    
    try:
        metadata = response.json()
    except json.JSONDecodeError:
        print("Cannot load", url)
        continue

    try:
        drama_change_rate = compute_change_rate(metadata)
    except:
        print("----> Cannot compute drama_change_rate for:", url)
        drama_change_rate = None
    try:
        final_scene_size = compute_final_scene_size(metadata)
    except:
        print("----> Cannot compute final_scene_size for:", url)
        final_scene_size = None
    try:
        all_in_index = compute_all_in_index(metadata)
    except:
        print("----> Cannot compute all_in_index for:", url)
        all_in_index = None

    new_row = {
        'all_in_index': all_in_index,
        'final_scene_size': final_scene_size,
        'drama_change_rate': drama_change_rate
    }

    extra_features_2 = extra_features_2.append(new_row, ignore_index=True)
extra_features_2

Computing Trilcke et al. 2017 metrics for http://localhost:8088/api/v1/corpora/em/plays/abeille-argelie
Computing Trilcke et al. 2017 metrics for http://localhost:8088/api/v1/corpora/em/plays/alarcon-verdad-sospechosa
Computing Trilcke et al. 2017 metrics for http://localhost:8088/api/v1/corpora/em/plays/amenta-somiglianza
Computing Trilcke et al. 2017 metrics for http://localhost:8088/api/v1/corpora/em/plays/amescua-esclavo-demonio
Computing Trilcke et al. 2017 metrics for http://localhost:8088/api/v1/corpora/em/plays/andreini-lo-schiavetto


/tmp/ipykernel_29872/941215788.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features_2 = extra_features_2.append(new_row, ignore_index=True)
/tmp/ipykernel_29872/941215788.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features_2 = extra_features_2.append(new_row, ignore_index=True)
/tmp/ipykernel_29872/941215788.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features_2 = extra_features_2.append(new_row, ignore_index=True)
/tmp/ipykernel_29872/941215788.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features_2 = extra_features_2.append(new_row, ignore_index=True)
/tmp/ipykernel_29872/941

Computing Trilcke et al. 2017 metrics for http://localhost:8088/api/v1/corpora/em/plays/anonym-lustig-pickelherings-spiel
Computing Trilcke et al. 2017 metrics for http://localhost:8088/api/v1/corpora/em/plays/anonym-macht-kleines-knaben
Computing Trilcke et al. 2017 metrics for http://localhost:8088/api/v1/corpora/em/plays/artieda-los-amantes
Computing Trilcke et al. 2017 metrics for http://localhost:8088/api/v1/corpora/em/plays/bailey-the-spiteful-sister
Computing Trilcke et al. 2017 metrics for http://localhost:8088/api/v1/corpora/em/plays/bances-esclavo-grillos
Computing Trilcke et al. 2017 metrics for http://localhost:8088/api/v1/corpora/em/plays/barbieri-l-inavertito
Computing Trilcke et al. 2017 metrics for http://localhost:8088/api/v1/corpora/em/plays/baro-clorise


/tmp/ipykernel_29872/941215788.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features_2 = extra_features_2.append(new_row, ignore_index=True)
/tmp/ipykernel_29872/941215788.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features_2 = extra_features_2.append(new_row, ignore_index=True)
/tmp/ipykernel_29872/941215788.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features_2 = extra_features_2.append(new_row, ignore_index=True)
/tmp/ipykernel_29872/941215788.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features_2 = extra_features_2.append(new_row, ignore_index=True)
/tmp/ipykernel_29872/941

Computing Trilcke et al. 2017 metrics for http://localhost:8088/api/v1/corpora/em/plays/baron-mirza
Computing Trilcke et al. 2017 metrics for http://localhost:8088/api/v1/corpora/em/plays/barry-ram-alley
Computing Trilcke et al. 2017 metrics for http://localhost:8088/api/v1/corpora/em/plays/bartolommei-amore-opera-a-caso
Computing Trilcke et al. 2017 metrics for http://localhost:8088/api/v1/corpora/em/plays/belyard-guisien
Computing Trilcke et al. 2017 metrics for http://localhost:8088/api/v1/corpora/em/plays/bienvenu-monde-malait
Computing Trilcke et al. 2017 metrics for http://localhost:8088/api/v1/corpora/em/plays/bonicelli-pantalone-bullo
Computing Trilcke et al. 2017 metrics for http://localhost:8088/api/v1/corpora/em/plays/braunschweig-fleischhawer


...

Computing Trilcke et al. 2017 metrics for http://localhost:8088/api/v1/corpora/em/plays/viau-pasiphae
Computing Trilcke et al. 2017 metrics for http://localhost:8088/api/v1/corpora/em/plays/viau-pyrame
Computing Trilcke et al. 2017 metrics for http://localhost:8088/api/v1/corpora/em/plays/virey-machabee
Cannot compute drama_change_rate for: http://localhost:8088/api/v1/corpora/em/plays/virey-machabee
Computing Trilcke et al. 2017 metrics for http://localhost:8088/api/v1/corpora/em/plays/virues-la-gran-semiramis
Computing Trilcke et al. 2017 metrics for http://localhost:8088/api/v1/corpora/em/plays/weise-baeurischer-machiavellus
Computing Trilcke et al. 2017 metrics for http://localhost:8088/api/v1/corpora/em/plays/weise-masaniello


/tmp/ipykernel_29872/941215788.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features_2 = extra_features_2.append(new_row, ignore_index=True)
/tmp/ipykernel_29872/941215788.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features_2 = extra_features_2.append(new_row, ignore_index=True)
/tmp/ipykernel_29872/941215788.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features_2 = extra_features_2.append(new_row, ignore_index=True)
/tmp/ipykernel_29872/941215788.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features_2 = extra_features_2.append(new_row, ignore_index=True)


Computing Trilcke et al. 2017 metrics for http://localhost:8088/api/v1/corpora/em/plays/weise-ungleich-liebesalliance
Computing Trilcke et al. 2017 metrics for http://localhost:8088/api/v1/corpora/em/plays/woodes-the-conflict-of-conscience
Computing Trilcke et al. 2017 metrics for http://localhost:8088/api/v1/corpora/em/plays/zamora-cada-uno
Computing Trilcke et al. 2017 metrics for http://localhost:8088/api/v1/corpora/em/plays/zamora-ser-fino-y-no-parecerlo
Computing Trilcke et al. 2017 metrics for http://localhost:8088/api/v1/corpora/em/plays/zayas-traicion-amistad


/tmp/ipykernel_29872/941215788.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features_2 = extra_features_2.append(new_row, ignore_index=True)
/tmp/ipykernel_29872/941215788.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features_2 = extra_features_2.append(new_row, ignore_index=True)


,all_in_index,final_scene_size,drama_change_rate
0,0.379310,0.375000,0.646429
1,0.926829,0.571429,0.639167
2,0.194030,0.769231,0.629455
3,1.000000,0.782609,0.417857
4,0.869565,0.625000,0.662172
...,...,...,...
145,0.933333,0.196078,0.749938
146,1.000000,0.055556,0.773810
147,1.000000,0.393939,0.701669
148,0.736842,0.371429,0.797895


In [19]:
'''
# For debugging purposes. Known errors:
# DIVISION BY ZERO --> the play has no scenes, only acts (segments = scenes)
# SPEAKERS --> a scene has no speakers, error in encoding or dumb scene
for url in errors:
  response = requests.get(url)
  data_json = json.loads(response.text)
  try:
    drama_change_rate = compute_change_rate(data_json)
  except Exception as e:
    print(url, e)
'''    

'\n# For debugging purposes. Known errors:\n# DIVISION BY ZERO --> the play has no scenes, only acts (segments = scenes)\n# SPEAKERS --> a scene has no speakers, error in encoding or dumb scene\nfor url in errors:\n  response = requests.get(url)\n  data_json = json.loads(response.text)\n  try:\n    drama_change_rate = compute_change_rate(data_json)\n  except Exception as e:\n    print(url, e)\n'

In [20]:
# add extra metrics to the table
#print(type(metadata_table))
#print(type(extra_features))
table_2 = pd.concat([table, extra_features_2],axis=1)
table_2.head()

,name,language,normalizedGenre,yearNormalized,size,averageClustering,density,averagePathLength,averageDegree,diameter,...,wordCountSp,wordCountStage,numOfP,numOfL,avg_betweenness,avg_closeness,avg_eigenvector,all_in_index,final_scene_size,drama_change_rate
0,abeille-argelie,FRE,Tragedy,1674,8,0.822024,0.607143,1.392857,4.250000,2,...,15962,0,0,1884,0.065476,0.741698,0.338143,0.379310,0.375000,0.646429
1,alarcon-verdad-sospechosa,ESP,Comedy,1619,14,0.769965,0.505495,1.593407,6.571429,3,...,16461,473,1,3306,0.049451,0.660847,0.243120,0.926829,0.571429,0.639167
2,amenta-somiglianza,ITA,Comedy,1706,13,0.903280,0.884615,1.115385,10.615385,2,...,33743,1242,2375,0,0.010490,0.904903,0.275634,0.194030,0.769231,0.629455
3,amescua-esclavo-demonio,ESP,0,1605,23,0.931742,0.869565,1.130435,19.130435,2,...,17413,742,0,3516,0.006211,0.899261,0.205852,1.000000,0.782609,0.417857
4,andreini-lo-schiavetto,ITA,Comedy,1612,24,0.859995,0.612319,1.391304,14.083333,3,...,38458,258,1659,40,0.017787,0.739950,0.191526,0.869565,0.625000,0.662172


In [53]:
table_2.shape

(150, 32)

#### 2. Adding metrics from Szemes and Vida 2022

In [21]:
# TEST -- get all type of useful metadata through the API
'''
metadata_url = "http://localhost:8088/api/v1/corpora/em/plays/cecchini-l-amico-tradito"
response1 = urlopen(metadata_url)
metadata = json.loads(response1.read())

spoken_text_url = "http://localhost:8088/api/v1/corpora/em/plays/torelli-la-merope/spoken-text-by-character"
spoken_text = requests.get(spoken_text_url).json()

network_metrics_url = "http://localhost:8088/api/v1/corpora/em/plays/torelli-la-merope/metrics"
response2 = urlopen(network_metrics_url)
network_metrics = json.loads(response2.read())

characters_url = "http://localhost:8088/api/v1/corpora/em/plays/torelli-la-merope/characters"
response3 = urlopen(characters_url)
character_metrics = json.loads(response3.read())
'''
# this will be wrapped in a function

'\nmetadata_url = "http://localhost:8088/api/v1/corpora/em/plays/cecchini-l-amico-tradito"\nresponse1 = urlopen(metadata_url)\nmetadata = json.loads(response1.read())\n\nspoken_text_url = "http://localhost:8088/api/v1/corpora/em/plays/torelli-la-merope/spoken-text-by-character"\nspoken_text = requests.get(spoken_text_url).json()\n\nnetwork_metrics_url = "http://localhost:8088/api/v1/corpora/em/plays/torelli-la-merope/metrics"\nresponse2 = urlopen(network_metrics_url)\nnetwork_metrics = json.loads(response2.read())\n\ncharacters_url = "http://localhost:8088/api/v1/corpora/em/plays/torelli-la-merope/characters"\nresponse3 = urlopen(characters_url)\ncharacter_metrics = json.loads(response3.read())\n'

In [22]:
'''Average Character Per Scene: First, the average number of characters in one scene is calculated. To make texts of different
lengths comparable, this number is divided by the number of scenes.'''

def compute_averageCharinScene(metadata):
    from statistics import mean
    avg_char_scenes = []
    if len(metadata['segments']) > 1:
        if 'segments' in metadata:
            for segment in metadata['segments']:
                #print(len(segment['speakers']))
                if 'speakers' in segment:
                    avg_char_scenes.append(len(segment['speakers']))
                else:
                    avg_char_scenes.append(0)
    else: #deals with einakter
        avg_char_scenes.append(len(metadata['characters']))
    #print(avg_char_scenes)
    averageCharinScene = mean(avg_char_scenes)
    #averageCharinScene = mean / len(metadata['segments'])      # why dividing by the number of scenes? it's already an average
    averageCharinScene = round(averageCharinScene)

    return averageCharinScene

In [23]:
#compute_averageCharinScene(metadata)

In [24]:
'''Average Degree – Max Degree Ratio: Degree is the number of nodes that a node is connected to via edges. From this,
the average and maximum degree for the network as a whole can be calculated. This is done so that statements could be made
that pertain to the network itself. This measure therefore gives the proportion of the average degree number relative to the
maximum degree number, and in a sense resembles density – to what extent this measure can be considered an independent
characteristic that describes networks from a unique point of view will be discussed later.'''

def compute_avg_max_degree_ratio(network_metrics):
  avg_max_degree_ratio = network_metrics['averageDegree'] / network_metrics['maxDegree']
  return avg_max_degree_ratio

In [25]:
#compute_avg_max_degree_ratio(network_metrics)

In [26]:
'''Maximum Degree – Number Character Ratio: To get the total number of characters correct, one has to be subtracted
from the actual number of characters, since a node does not interact with itself. The ratio shows what fraction of the entire
cast does the node with the highest degree interact with.'''

def compute_max_degree_num_char_ratio(network_metrics):
  num_chars = len(network_metrics['nodes'])-1
  max_degree_num_char_ratio = network_metrics['maxDegree'] / num_chars

  return max_degree_num_char_ratio

In [27]:
#compute_max_degree_num_char_ratio(network_metrics)

In [28]:
'''High Weigthed Degree, Medium Weigthed Degree, Low Weigthed Degree: The same method was employed to calculate the weighted degree
of characters as nodes. Here, the proportion of characters with many, medium or few connections in the network was
examined. Since the result is always adjusted to the number of characters, this measure is also size-independent, which allows us
to compare works with different numbers of characters. Our previous measures showed that the metrics of weighted degree (which
takes into account not only the number of edges leading into a node but also their “weight”, in this case the frequency of
appearing together in the same scene) better captures the differences between genres and the specific characteristics of plays
than using only the degree for the calculations.'''

def compute_weighted_degree_distribution(network_metrics):

    degrees = []
    for el in network_metrics['nodes']:
      degrees.append(el['weightedDegree'])
    degrees.sort()
    #print(degrees)

    low_degree = []
    medium_degree = []
    high_degree = []
    for el in network_metrics['nodes']:
      wgh_degree = el['weightedDegree']
      if wgh_degree < np.percentile(degrees, 33):      # using percentiles instead of kmeans, it's still size-independent
        low_degree.append(wgh_degree)
      elif wgh_degree < np.percentile(degrees, 66):
        medium_degree.append(wgh_degree)
      else:
        high_degree.append(wgh_degree)

    # compute distribution

    total_characters = len(network_metrics['nodes'])
    share_low = len(low_degree) / total_characters
    share_medium = len(medium_degree) / total_characters
    share_high = len(high_degree) / total_characters

    return share_low, share_medium, share_high

In [29]:
#compute_weighted_degree_distribution(network_metrics)

In [30]:
'''High Speech, Medium Speech, Low Speech: [...] k-means clustering was employed (k=3, likewise) to distinguish between characters
in plays who are not just members of a group in the stage (for which DraCor uses the annotation “isGroup = True”, e.g. “street vendors”).
The three groups were then ranked along their mean value, and the number of characters in each category was then adjusted to the
total number of non-group characters in the play. This ratio makes it possible to compare plays with different numbers of characters,
since it shows not only the number of characters who speak a lot/medium/less in a text, but also the proportion of characters who
do so, which essentially gives us an idea of the distribution of linguistic complexity among characters. '''

def compute_speech_intensity(spoken_text):
    speech_length = []
    for character in spoken_text:
      char_speech = len(re.findall(r'\w+', str(character['text'])))
      speech_length.append(char_speech)
    speech_length.sort()
    #print("SL", speech_length)

    lowSpeech = 0
    mediumSpeech = 0
    highSpeech = 0
    from statistics import mean
    for character in spoken_text:
      char_speech = len(re.findall(r'\w+', str(character['text'])))
      #if character['isGroup'] == False:         # why excluding them? they might be reintegrated
      #print(character['id'], len(re.findall(r'\w+', str(character['text']))))
      if char_speech < np.percentile(speech_length, 33):      # using percentiles instead of kmeans, it's still size-independent
        lowSpeech += 1
      elif char_speech < np.percentile(speech_length, 66):
        mediumSpeech += 1
      else:
        highSpeech += 1

    # compute distribution

    total_characters = len(spoken_text)
    share_low = lowSpeech / total_characters
    share_medium = mediumSpeech / total_characters
    share_high = highSpeech / total_characters

    return share_low, share_medium, share_high

In [31]:
#compute_speech_intensity(spoken_text)

In [32]:
'''Average Character Speech: This shows the average number of words spoken by a character
in a play. In Szemes/Vidas implementation, only characters who speak more than 10 times were considered,
but we decided to consider all of them.'''

def compute_averageCharSpeech(character_metrics):
    avg_list = []
    for character in character_metrics:
        #if character["numOfSpeechActs"] < 10:  # Skip characters with less than 10 lines
        #    continue
        if character["numOfSpeechActs"] == 0:
            character["numOfSpeechActs"] = 1
        avg_char = character["numOfWords"]/character["numOfSpeechActs"]  
        #print(character["id"], avg_char)
        avg_list.append(avg_char)
    averageCharSpeech = mean(avg_list)

    return averageCharSpeech

In [33]:
#compute_averageCharSpeech(character_metrics)

In [34]:
def load_all_metadata(slug):
    try:
        metadata_url = f"http://localhost:8088/api/v1/corpora/em/plays/{slug}"
        response1 = urlopen(metadata_url)
        metadata = json.loads(response1.read())
    except Exception as e:
        print("Cannot load metadata for", slug, e)
        metadata = None

    try:
        spoken_text_url = f"http://localhost:8088/api/v1/corpora/em/plays/{slug}/spoken-text-by-character"
        spoken_text = requests.get(spoken_text_url).json()
    except Exception as e:
        print("Cannot load spoken text for", slug, e)
        spoken_text = None

    try:
        network_metrics_url = f"http://localhost:8088/api/v1/corpora/em/plays/{slug}/metrics"
        response2 = urlopen(network_metrics_url)
        network_metrics = json.loads(response2.read())
    except Exception as e:
        print("Cannot load network metrics for", slug, e)
        network_metrics = None

    try:
        character_url = f"http://localhost:8088/api/v1/corpora/em/plays/{slug}/characters"
        character_metrics = requests.get(character_url).json()
    except Exception as e:
        print("Cannot load character metrics for", slug, e)
        character_metrics = None

    return metadata, spoken_text, network_metrics, character_metrics


In [70]:
# prepare empty columns

data = {'averageCharinScene': [],
        'avg_max_degree_ratio': [],
        'max_degree_num_char_ratio': [],
        'lowWeightedDegree': [],
        'mediumWeightedDegree': [],
        'highWeightedDegree': [],
        'averageCharSpeech':[],
        'lowSpeech': [],
        'mediumSpeech': [],
        'highSpeech': []}

extra_features_3 = pd.DataFrame(data)
extra_features_3

,averageCharinScene,avg_max_degree_ratio,max_degree_num_char_ratio,lowWeightedDegree,mediumWeightedDegree,highWeightedDegree,averageCharSpeech,lowSpeech,mediumSpeech,highSpeech


In [36]:
# debug
'''
l = ['baron-mirza']
for slug in l:
    response = load_all_metadata(slug)
    metadata = response[0]
    spoken_text = response[1]
    network_metrics = response[2]
    character_metrics = response[3]
  #except Exception as e:
   #   print("This play throws an error:", url, e)

    averageCharSpeech = compute_averageCharSpeech(character_metrics)
    print(slug, averageCharSpeech)
'''    

'\nl = [\'baron-mirza\']\nfor slug in l:\n    response = load_all_metadata(slug)\n    metadata = response[0]\n    spoken_text = response[1]\n    network_metrics = response[2]\n    character_metrics = response[3]\n  #except Exception as e:\n   #   print("This play throws an error:", url, e)\n\n    averageCharSpeech = compute_averageCharSpeech(character_metrics)\n    print(slug, averageCharSpeech)\n'

In [71]:
counter = 1
for slug in metadata_table['name'].tolist():
    print("Loading", slug, "...")
    try:
        response = load_all_metadata(slug)
        metadata = response[0]
        spoken_text = response[1]
        network_metrics = response[2]
        character_metrics = response[3]
    except Exception as e:
        print("Metadata could not be loaded properly for:", slug, e)
        continue

    try:
        averageCharinScene = compute_averageCharinScene(metadata)
    except Exception as e:
        print("----> Cannot compute averageCharinScene for:", slug, e)
        averageCharinScene = None
    
    try:
        avg_max_degree_ratio = compute_avg_max_degree_ratio(network_metrics)
    except Exception as e:
        print("----> Cannot compute avg_max_degree_ratio for:", slug, e)
        avg_max_degree_ratio = None
    
    try:
        max_degree_num_char_ratio = compute_max_degree_num_char_ratio(network_metrics)
    except Exception as e:
        print("----> Cannot compute max_degree_num_char_ratio for:", slug, e)
        max_degree_num_char_ratio = None
    
    try:
        lowWeightedDegree, mediumWeightedDegree, highWeightedDegree = compute_weighted_degree_distribution(network_metrics)
    except Exception as e:
        print("----> Cannot compute weighted_degree_distribution for:", slug, e)
        lowWeightedDegree = mediumWeightedDegree = highWeightedDegree = None
    
    try:
        lowSpeech, mediumSpeech, highSpeech = compute_speech_intensity(spoken_text)
    except Exception as e:
        print("----> Cannot compute speech_intensity for:", slug, e)
        lowSpeech = mediumSpeech = highSpeech = None
    
    try:
        averageCharSpeech = compute_averageCharSpeech(character_metrics)
    except Exception as e:
        print("----> Cannot compute averageCharSpeech for:", slug, e)
        averageCharSpeech = None

    new_row = {
        'averageCharinScene': averageCharinScene,
        'avg_max_degree_ratio': avg_max_degree_ratio,
        'max_degree_num_char_ratio': max_degree_num_char_ratio,
        'lowWeightedDegree': lowWeightedDegree,
        'mediumWeightedDegree': mediumWeightedDegree,
        'highWeightedDegree': highWeightedDegree,
        'averageCharSpeech': averageCharSpeech,
        'lowSpeech': lowSpeech,
        'mediumSpeech': mediumSpeech,
        'highSpeech': highSpeech
    }

    extra_features_3 = extra_features_3.append(new_row, ignore_index=True)

    print('Computed all Szemes/Vida metrics for', slug, counter, "/150")
    counter += 1

extra_features_3


Loading... abeille-argelie


/tmp/ipykernel_29872/3735916490.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features_3 = extra_features_3.append(new_row, ignore_index=True)


Computed all Szemes/Vida metrics for abeille-argelie 1 /150
Loading... alarcon-verdad-sospechosa


/tmp/ipykernel_29872/3735916490.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features_3 = extra_features_3.append(new_row, ignore_index=True)


Computed all Szemes/Vida metrics for alarcon-verdad-sospechosa 2 /150
Loading... amenta-somiglianza


...

Computed all Szemes/Vida metrics for zamora-cada-uno 148 /150
Loading... zamora-ser-fino-y-no-parecerlo


/tmp/ipykernel_29872/3735916490.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features_3 = extra_features_3.append(new_row, ignore_index=True)


Computed all Szemes/Vida metrics for zamora-ser-fino-y-no-parecerlo 149 /150
Loading... zayas-traicion-amistad
Computed all Szemes/Vida metrics for zayas-traicion-amistad 150 /150


/tmp/ipykernel_29872/3735916490.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features_3 = extra_features_3.append(new_row, ignore_index=True)


,averageCharinScene,avg_max_degree_ratio,max_degree_num_char_ratio,lowWeightedDegree,mediumWeightedDegree,highWeightedDegree,averageCharSpeech,lowSpeech,mediumSpeech,highSpeech
0,2.0,0.607143,1.000000,0.375000,0.250000,0.375000,56.125942,0.375000,0.250000,0.375000
1,3.0,0.597403,0.846154,0.357143,0.285714,0.357143,17.281830,0.357143,0.285714,0.357143
2,3.0,0.884615,1.000000,0.307692,0.307692,0.384615,13.053104,0.307692,0.307692,0.384615
3,16.0,0.869565,1.000000,0.347826,0.260870,0.391304,18.021854,0.347826,0.304348,0.347826
4,4.0,0.640152,0.956522,0.333333,0.333333,0.333333,19.410673,0.333333,0.333333,0.333333
...,...,...,...,...,...,...,...,...,...,...
145,3.0,0.542986,0.520000,0.313725,0.333333,0.352941,26.834867,0.333333,0.313725,0.352941
146,3.0,0.317460,0.823529,0.333333,0.277778,0.388889,112.493715,0.333333,0.333333,0.333333
147,8.0,0.554433,0.843750,0.181818,0.484848,0.333333,9.493720,0.333333,0.333333,0.333333
148,7.0,0.499048,0.882353,0.285714,0.371429,0.342857,13.063249,0.342857,0.314286,0.342857


In [38]:
# virey-machabee is the only play raising problems here
# it seems the json is malformed, but all three metadata files can be read separately without problems

In [72]:
table_3 = pd.concat([table_2, extra_features_3],axis=1)
table_3.head(6)

,name,language,normalizedGenre,yearNormalized,size,averageClustering,density,averagePathLength,averageDegree,diameter,...,averageCharinScene,avg_max_degree_ratio,max_degree_num_char_ratio,lowWeightedDegree,mediumWeightedDegree,highWeightedDegree,averageCharSpeech,lowSpeech,mediumSpeech,highSpeech
0,abeille-argelie,FRE,Tragedy,1674,8,0.822024,0.607143,1.392857,4.250000,2,...,2.0,0.607143,1.000000,0.375000,0.250000,0.375000,56.125942,0.375000,0.250000,0.375000
1,alarcon-verdad-sospechosa,ESP,Comedy,1619,14,0.769965,0.505495,1.593407,6.571429,3,...,3.0,0.597403,0.846154,0.357143,0.285714,0.357143,17.281830,0.357143,0.285714,0.357143
2,amenta-somiglianza,ITA,Comedy,1706,13,0.903280,0.884615,1.115385,10.615385,2,...,3.0,0.884615,1.000000,0.307692,0.307692,0.384615,13.053104,0.307692,0.307692,0.384615
3,amescua-esclavo-demonio,ESP,0,1605,23,0.931742,0.869565,1.130435,19.130435,2,...,16.0,0.869565,1.000000,0.347826,0.260870,0.391304,18.021854,0.347826,0.304348,0.347826
4,andreini-lo-schiavetto,ITA,Comedy,1612,24,0.859995,0.612319,1.391304,14.083333,3,...,4.0,0.640152,0.956522,0.333333,0.333333,0.333333,19.410673,0.333333,0.333333,0.333333
5,anonym-lustig-pickelherings-spiel,GER,Comedy,1620,3,1.000000,1.000000,1.000000,2.000000,1,...,3.0,1.000000,1.000000,0.000000,0.000000,1.000000,30.576282,0.333333,0.333333,0.333333


In [73]:
table_3.shape

(150, 42)

#### 3. Adding metrics from Algee-Hewitt 2017

In [74]:
# calculating the distribution of eigenvector centrality

# Gini coefficient
# modified from https://planspace.org/2013/06/21/how-to-calculate-gini-coefficient-from-raw-data-in-python/
def gini(network_metrics):
    eigenvectors = []
    for char in network_metrics['nodes']:
      eigenvectors.append(char['eigenvector']) # creates eigenvectors list
    #print(eigenvectors)
    sorted_list = sorted(eigenvectors)
    height, area = 0, 0
    for value in sorted_list:
        height += value
        area += height - value / 2.
    fair_area = height * len(eigenvectors) / 2.
    gini_coefficient = (fair_area - area) / fair_area

    return gini_coefficient

In [75]:
'''
For a single metric based on EC, then, we can combine these two measurements, scaling the Gini coefficient
by dividing it by the percentage of characters in the top quartile of the eigenvector distribution and
creating a ratio between them. [...] Moreover, because both of these metrics offer information on the
function of the protagonist (whether the strength of the core/periphery divide, or the distribution
of protagonists within the core), their combination in this ratio yields a measurement that I call
“protagonism.”
'''
def compute_protagonism(network_metrics):
    counter_chars = 0
    counter_top_quartile = 0
    eigenvectors = []
    from statistics import mean
    for char in network_metrics['nodes']:
      counter_chars += 1
      eigenvectors.append(char['eigenvector']) # creates eigenvectors list
      egv = char['eigenvector']
      if egv > np.percentile(eigenvectors, 75):      # top quartile
          counter_top_quartile +=1
    print("Characters", counter_chars)
    print("Characters in top 25%", counter_top_quartile)
    gini_coefficient = gini(network_metrics)
    percent_top_25 = (counter_top_quartile*100) / counter_chars
    print("Gini", gini_coefficient)
    print('Percent', percent_top_25)
    try:
        protagonism = gini_coefficient / percent_top_25
    except: # deals with e.g. fully-connected plays, where Gini = 0 and all egv are equal
        protagonism = 0

    return protagonism

In [ ]:
# test
'''
network_metrics_url = "http://localhost:8088/api/corpora/em/play/barbieri-l-inavertito/metrics"
response2 = urlopen(network_metrics_url)
network_metrics = json.loads(response2.read())
compute_protagonism(network_metrics)
'''

In [76]:
# mediateness = maximum normalized betweenness of a play’s characters

def compute_mediateness(network_metrics):
    betweenesses = []
    for char in network_metrics['nodes']:
      betweenesses.append(char['betweenness'])
    betweenesses = preprocessing.normalize([betweenesses])
    mediateness = betweenesses.max()

    return(mediateness)

# what is meant here by normalisation?

Results in my implementation are very different from Algee-Hewitt's... Why?

1) he didn't worked with the same (clean) DraCor data

2) it is not 100% clear how he calculated protagonism and mediateness -- mine is an approximation

In [77]:
# prepare empty columns

data = {'protagonism': [],
        'mediateness': []}

extra_features_4 = pd.DataFrame(data)
extra_features_4

,protagonism,mediateness


In [78]:
counter = 1
for slug in metadata_table['name']:
  try:
      response = load_all_metadata(slug)
      metadata = response[0]
      spoken_text = response[1]
      network_metrics = response[2]
  except Exception as e:
      print("This play throws an error:", url, e)

  protagonism = compute_protagonism(network_metrics)
  mediateness = compute_mediateness(network_metrics)

  print('Computed all Algee-Hewitt metrics for', slug, counter, "/150")
  counter += 1

  new_row = {'protagonism': protagonism,
        'mediateness': mediateness}

  extra_features_4 = extra_features_4.append(new_row, ignore_index=True)

extra_features_4

Characters 8
Characters in top 25% 2
Gini 0.17016935884119036
Percent 25.0
Computed all Algee-Hewitt metrics for abeille-argelie 1 /150


/tmp/ipykernel_29872/3944290659.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features_4 = extra_features_4.append(new_row, ignore_index=True)


Characters 14
Characters in top 25% 2
Gini 0.25059604639947847
Percent 14.285714285714286
Computed all Algee-Hewitt metrics for alarcon-verdad-sospechosa 2 /150


/tmp/ipykernel_29872/3944290659.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features_4 = extra_features_4.append(new_row, ignore_index=True)


Characters 13
Characters in top 25% 3
Gini 0.05774440245097504
Percent 23.076923076923077
Computed all Algee-Hewitt metrics for amenta-somiglianza 3 /150


...

Characters 35
Characters in top 25% 9
Gini 0.22920827477952224
Percent 25.714285714285715
Computed all Algee-Hewitt metrics for zamora-ser-fino-y-no-parecerlo 149 /150


/tmp/ipykernel_29872/3944290659.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features_4 = extra_features_4.append(new_row, ignore_index=True)


Characters 13
Characters in top 25% 1
Gini 0.030803228897376616
Percent 7.6923076923076925
Computed all Algee-Hewitt metrics for zayas-traicion-amistad 150 /150


/tmp/ipykernel_29872/3944290659.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_features_4 = extra_features_4.append(new_row, ignore_index=True)


,protagonism,mediateness
0,0.006807,0.891631
1,0.017542,0.549356
2,0.002502,0.412395
3,0.016973,0.329647
4,0.004265,0.668928
...,...,...
145,0.014100,0.474680
146,0.027683,0.991477
147,0.007022,0.422613
148,0.008914,0.523328


In [79]:
final_table = pd.concat([table_3, extra_features_4],axis=1)
final_table.head(6)

,name,language,normalizedGenre,yearNormalized,size,averageClustering,density,averagePathLength,averageDegree,diameter,...,max_degree_num_char_ratio,lowWeightedDegree,mediumWeightedDegree,highWeightedDegree,averageCharSpeech,lowSpeech,mediumSpeech,highSpeech,protagonism,mediateness
0,abeille-argelie,FRE,Tragedy,1674,8,0.822024,0.607143,1.392857,4.250000,2,...,1.000000,0.375000,0.250000,0.375000,56.125942,0.375000,0.250000,0.375000,0.006807,0.891631
1,alarcon-verdad-sospechosa,ESP,Comedy,1619,14,0.769965,0.505495,1.593407,6.571429,3,...,0.846154,0.357143,0.285714,0.357143,17.281830,0.357143,0.285714,0.357143,0.017542,0.549356
2,amenta-somiglianza,ITA,Comedy,1706,13,0.903280,0.884615,1.115385,10.615385,2,...,1.000000,0.307692,0.307692,0.384615,13.053104,0.307692,0.307692,0.384615,0.002502,0.412395
3,amescua-esclavo-demonio,ESP,0,1605,23,0.931742,0.869565,1.130435,19.130435,2,...,1.000000,0.347826,0.260870,0.391304,18.021854,0.347826,0.304348,0.347826,0.016973,0.329647
4,andreini-lo-schiavetto,ITA,Comedy,1612,24,0.859995,0.612319,1.391304,14.083333,3,...,0.956522,0.333333,0.333333,0.333333,19.410673,0.333333,0.333333,0.333333,0.004265,0.668928
5,anonym-lustig-pickelherings-spiel,GER,Comedy,1620,3,1.000000,1.000000,1.000000,2.000000,1,...,1.000000,0.000000,0.000000,1.000000,30.576282,0.333333,0.333333,0.333333,0.000000,0.000000


In [81]:
final_table

,name,language,normalizedGenre,yearNormalized,size,averageClustering,density,averagePathLength,averageDegree,diameter,...,max_degree_num_char_ratio,lowWeightedDegree,mediumWeightedDegree,highWeightedDegree,averageCharSpeech,lowSpeech,mediumSpeech,highSpeech,protagonism,mediateness
0,abeille-argelie,FRE,Tragedy,1674,8,0.822024,0.607143,1.392857,4.250000,2,...,1.000000,0.375000,0.250000,0.375000,56.125942,0.375000,0.250000,0.375000,0.006807,0.891631
1,alarcon-verdad-sospechosa,ESP,Comedy,1619,14,0.769965,0.505495,1.593407,6.571429,3,...,0.846154,0.357143,0.285714,0.357143,17.281830,0.357143,0.285714,0.357143,0.017542,0.549356
2,amenta-somiglianza,ITA,Comedy,1706,13,0.903280,0.884615,1.115385,10.615385,2,...,1.000000,0.307692,0.307692,0.384615,13.053104,0.307692,0.307692,0.384615,0.002502,0.412395
3,amescua-esclavo-demonio,ESP,0,1605,23,0.931742,0.869565,1.130435,19.130435,2,...,1.000000,0.347826,0.260870,0.391304,18.021854,0.347826,0.304348,0.347826,0.016973,0.329647
4,andreini-lo-schiavetto,ITA,Comedy,1612,24,0.859995,0.612319,1.391304,14.083333,3,...,0.956522,0.333333,0.333333,0.333333,19.410673,0.333333,0.333333,0.333333,0.004265,0.668928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,weise-ungleich-liebesalliance,GER,Comedy,1708,51,0.707500,0.282353,2.069020,14.117647,4,...,0.520000,0.313725,0.333333,0.352941,26.834867,0.333333,0.313725,0.352941,0.014100,0.474680
146,woodes-the-conflict-of-conscience,ENG,Comedy,1572,18,0.705047,0.261438,1.741667,4.444444,3,...,0.823529,0.333333,0.277778,0.388889,112.493715,0.333333,0.333333,0.333333,0.027683,0.991477
147,zamora-cada-uno,ESP,0,1708,33,0.825678,0.467803,1.573864,14.969697,3,...,0.843750,0.181818,0.484848,0.333333,9.493720,0.333333,0.333333,0.333333,0.007022,0.422613
148,zamora-ser-fino-y-no-parecerlo,ESP,0,1701,35,0.823315,0.440336,1.579832,14.971429,3,...,0.882353,0.285714,0.371429,0.342857,13.063249,0.342857,0.314286,0.342857,0.008914,0.523328


In [82]:
none_count = final_table.isna().sum().sum()
print("Total number of None/NaN values in the DataFrame:", none_count)

Total number of None/NaN values in the DataFrame: 27


In [ ]:
# we need to address none values, since they would mess up the vector 

In [101]:
df = final_table.copy()

In [93]:
# addressing the none values 
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
def fill_na_with_mean_around_year(row, df):
    year = row['yearNormalized']
    window_start = year - 15
    window_end = year + 15
    mask = (df['yearNormalized'] >= window_start) & (df['yearNormalized'] <= window_end)
    mean_values = df.loc[mask, :].mean()
    filled_row = row.fillna(mean_values)
    for col in df.columns:
        if pd.isna(row[col]) and not pd.isna(filled_row[col]):
            print(f" The metric '{col}' for '{row['name']}' is NaN. We filled it with value '{filled_row[col]}'. "
                  f"The play is from {year}, so we took data from {window_start} to {window_end} to compute the mean.")
    return filled_row

# Apply the function to each row
df_filled = df.apply(lambda row: fill_na_with_mean_around_year(row, df), axis=1)

 The metric 'lowSpeech' for 'bailey-the-spiteful-sister' is NaN. We filled it with value '0.34162035083406556'. The play is from 1667, so we took data from 1652 to 1682 to compute the mean.
 The metric 'mediumSpeech' for 'bailey-the-spiteful-sister' is NaN. We filled it with value '0.3059520804228365'. The play is from 1667, so we took data from 1652 to 1682 to compute the mean.
 The metric 'highSpeech' for 'bailey-the-spiteful-sister' is NaN. We filled it with value '0.352427568743098'. The play is from 1667, so we took data from 1652 to 1682 to compute the mean.
 The metric 'lowSpeech' for 'baron-mirza' is NaN. We filled it with value '0.33528413605166113'. The play is from 1655, so we took data from 1640 to 1670 to compute the mean.
 The metric 'mediumSpeech' for 'baron-mirza' is NaN. We filled it with value '0.3110501095150595'. The play is from 1655, so we took data from 1640 to 1670 to compute the mean.
 The metric 'highSpeech' for 'baron-mirza' is NaN. We filled it with value '0

In [102]:
# Group by 'language', compute mean for numeric columns, and fill NaN values
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns  # Select numeric columns

# Calculate means per language group
means_by_language = df.groupby('language')[numeric_columns].transform('mean')

# Create a new DataFrame df_filled with filled NaN values
df_filled = df.copy()  # Create a copy of df to preserve original
for col in numeric_columns:
    mask = df_filled[col].isna()  # Create a mask for NaN values in the column
    if mask.any():  # Proceed only if NaN values exist in the column
        df_filled.loc[mask, col] = means_by_language.loc[mask, col]  # Fill NaNs with the corresponding means
        # Print statement for each filled NaN value
        for slug in df_filled.loc[mask, 'name'].tolist():
            language = df_filled.loc[df_filled['name'] == slug, 'language'].iloc[0]
            filled_value = means_by_language.loc[mask, col].loc[df_filled['language'] == language].iloc[0]
            print(f"The metric '{col}' for '{slug}' is NaN. "
                  f"We filled it with value '{filled_value}'. The play is from {language}, "
                  f"so we took data from other {language} plays to compute the mean.")

# Save df_filled to a variable
df_filled_variable = df_filled


The metric 'all_in_index' for 'zamora-cada-uno' is NaN. We filled it with value '0.9027832813342138'. The play is from ESP, so we took data from other ESP plays to compute the mean.
The metric 'all_in_index' for 'zamora-ser-fino-y-no-parecerlo' is NaN. We filled it with value '0.9027832813342138'. The play is from ESP, so we took data from other ESP plays to compute the mean.
The metric 'all_in_index' for 'zayas-traicion-amistad' is NaN. We filled it with value '0.9027832813342138'. The play is from ESP, so we took data from other ESP plays to compute the mean.
The metric 'final_scene_size' for 'zamora-cada-uno' is NaN. We filled it with value '0.5431185110107231'. The play is from ESP, so we took data from other ESP plays to compute the mean.
The metric 'final_scene_size' for 'zamora-ser-fino-y-no-parecerlo' is NaN. We filled it with value '0.5431185110107231'. The play is from ESP, so we took data from other ESP plays to compute the mean.
The metric 'final_scene_size' for 'zayas-trai

In [103]:
df_filled_variable.plot.bar(x='name', y='averageCharSpeech',figsize=(100,80))

<Axes: xlabel='name'>

In [105]:
df_filled_variable.to_csv("EmDraCor_enriched_metadata.csv")

In [ ]:
# check which rows have null values

# Read CSV from URL
url = 'https://raw.githubusercontent.com/lucagiovannini7/emdracor/main/metadata/EmDraCor_enriched_metadata.csv'
df = pd.read_csv(url)

# Find and print cells with null values
counter = 0
for index, row in df.iterrows():
    for col in df.columns:
        if row[col] == 0.0:
            counter += 1
            #print(f"Metric ({col}) in work ({row['name']}) is 0.0")
print((415/6600)*100)

# Trashbin

In [ ]:
url = "http://localhost:8088/api/corpora/em/play/schoch-comoedia-studenten"
response = requests.get(url)
data_json = json.loads(response.text)
drama_change_rate = compute_change_rate(data_json)
#print(data_json['segments'][3]['speakers'])

In [ ]:
'''Average Character Speech: This shows the average number of words spoken by a character
in a play. Only characters who speak more than 10 times were considered.'''

def compute_averageCharSpeech(spoken_text):
    all_avg_num_words = []

    for character in spoken_text:
        if len(character['text']) < 10:  # Skip characters with less than 10 lines
            continue

        char_avg_num_words = []
        for line in character['text']:
            char_avg_num_words.append(len(re.findall(r'\w+', line)))

        mean_char = mean(char_avg_num_words)
        mean_char = round(mean_char)
        print(f"Character: {character['label']}, Average words per line: {mean_char}")
        all_avg_num_words.append(mean_char)

    averageCharSpeech = mean(all_avg_num_words)
    averageCharSpeech = round(averageCharSpeech, 2)

    return averageCharSpeech